In [4]:
import json
f = open('C:/Users/GOWRICH1/Downloads/annotations.json', encoding="utf8")
TRAIN_DATA = json.load(f)
f.close()

In [5]:
TRAIN_DATA

{'classes': ['ORG', 'DISEASE', 'DRUG', 'PERSON', 'HCP', 'MONEY', 'LOCATION'],
 'annotations': [['I will say this, talk to your neurologist. Medical marijuana helps me with sleep, appetite and spasms but they will probably put you on a DMD (disease modifying drug) to slow progression of the ms. There is no cure for ms, there is no magic pill or remedy. I have ms was on tecfiera for 2 years now im on ocrevus because tecfidera wasnt working. I have progressed to ppms and medical marijuana helps me alot. I have 5 lesions total on my frontal lobe, 3 on my temporal lobe and numerous periventricle lesions (explains why i walk like im drunk especially when overheated. This group is here for you. Come here to vent or ask questions. A ms diagnosis sucks but its not fatal. We all have to get used to out new selfs. Im a bobs burgers fan and in the words of Gene… “this is me now!” And CIS is clinically isolated syndrome, basically you had a episode and if it happens again you will be diagnosed with

In [6]:
import pandas as pd
import os
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

#nlp = spacy.blank("en") # load a new spacy model
nlp = spacy.load("en_ner_bc5cdr_md") # load other spacy model

db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA['annotations']): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

os.chdir(r'C:/Users/GOWRICH1/Downloads/NER Training')
db.to_disk("./train.spacy") # save the docbin object

C:\ProgramData\Anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.4.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.2.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
100%|█████████████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 119.28it/s]


In [8]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


[i] Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-06-07 11:44:58.465571: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-07 11:44:58.465624: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [9]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     97.73    0.00    0.00    0.00    0.00
 18     200       1939.69   3451.47   91.28   95.77   87.18    0.91
 38     400         53.32    139.23  100.00  100.00  100.00    1.00
 60     600         22.88     16.57  100.00  100.00  100.00    1.00
 84     800         33.00     22.12  100.00  100.00  100.00    1.00
108    1000         23.24     10.03  100.00  100.00  100.00    1.00
135    1200         71.41     27.18  100.00  100.00  100.00    1.00
163    1400         71.69     28.99  100.00  100.00  100.00    1.00
192    1600          0.05 

2022-06-07 11:46:22.801797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-06-07 11:46:22.801860: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[2022-06-07 11:46:44,447] [INFO] Set up nlp object from config
[2022-06-07 11:46:44,468] [INFO] Pipeline: ['tok2vec', 'ner']
[2022-06-07 11:46:44,477] [INFO] Created vocabulary
[2022-06-07 11:46:44,491] [INFO] Finished initializing nlp object
[2022-06-07 11:46:45,610] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [17]:
nlp1 = spacy.load(r"C:/Users/GOWRICH1/Downloads/NER Training/model-best") #load the best model
#doc = nlp1("Hey everyone, 22F, DX'd in February of this year, RRMS.When diagnosed, I mainly went to a doctor because I was having trouble completing my PhD homework. I couldn't process or read words, visually, without getting a terrible migraine. I was dizzy. Come to find out, the doctors found ""new, old and very old"" lesions. They were very deep, T1 and T2 white matter lesions on my brain. I also recently found out that I have a C4 lesion on my cervical spine. And you know how that goes, steroids and treatment. However, I haven't been the same since my first flare up in February. It takes me much longer to process things and to critically think. And that's on simple concepts. It's very hard for me to think of the right words at times, to remember, to learn. I often forget what I'm talking about. I went from working 4 jobs, doing research, going to classes, PLUS extracurriculars and a social life, to not remembering what I wanted to look up. Or not remembering the words I was so good at using. And I'm on a cane at 22yo, using a shower chair to help me wash up.My PhD program is kind so I'm taking the year off. But I think they expect me to come back. I've come to terms with the fact that life happens and maybe this wasn't the most fulfilling route for me. Especially since diagnosis, that was brought on by a big stressed breakdown I had, I've come to terms that maybe I wasn't meant for a career that involves me to research and present and teach. I would also have to move to a whole different state with my financial offer dependent on how I function as a TA, my academic performance, professor feedback, steps towards my thesis, etc. Honestly, I'd rather stay here at home. I'm making a step towards recovery (NP said I'm not totally there yet) and my insurance is mostly exclusive to my mom's state job. University insurance might require a copay for Ocrevus, plus the other medications I'm on. I'd just rather give myself a break from my perfectionist, workaholic attitude and focus on my health for once.Before I break the news to my department that I don't plan on moving to their state and continuing my studies, I want to have some careers in mind that are MS friendly. I have my Bachelor's with honors and awards (thank God), and I have TA experience, but I'm not sure what my fit is.So, rather than stress out about it right now, I wanna hear your stories, please! The negative, the positive, all of it. If you're willing, please share your career and how compatible it is with your MS.Thank you so much! Very much appreciated 🧡") # input sample text
doc=nlp1("I finished up the flimsy for Kathleen Tracy's Mini Valentine Mystery today. Nice and simple. Barb's is below though she added a roses border on hers and it is cute!I sewed with Becky a few days ago and convinced her to start Ruby Jubilee! She wasted no time in picking out fabrics and whipping these up. She was my blogging friend before I moved to Arizona and we lived close enough together out there to be in the same guild! You may remember we did the Quiltville cruise together and lots of hiking too. Now we sew together online now and then. SEW much fun! Barb finished hers up in greens. It is going to be so cool to see these three come together in three different colorways. The next issue of Quiltmaker is out digitally and so I know we have plenty of blocks to make in the next few months. I get it on Scribd which I have a free trial of. I just cannot figure out how to print from it (unless I screenshot and do it that way and it usually does not enlarge well when I try that) and though I have the patterns from Quiltmaker and could go that route, I am tempted to just wait for my actual issue to arrive. I have plenty to keep me busy!Recently finished these two books. I liked them both.I have been working on R Trail and so it is trimmed, stay stitched, aqua borders cut and added and final borders assembled. I am hoping to get at least two of them on tonight, more if I have the energy.In family and health news: my daughter, son in law and granddaughter are over covid. My oldest has it again (I think this is the third time) and it looks like my mom and dad have it too. My sister thinks she had it a week or two ago. It is crazy contagious and amazing Kathy and I have not caught it, but with all my medical appointments I feel like it won't be long. Hazel starts daycare next week so we may pick it up from her (and Hannah starts a new job too) so our exposure will be greater. I have enough issues without that on top of it. Not sure if I told you guys but my RRMS was changed to PPMS. They originally thought I had PPMS, then decided I did not and now the new neuros think that was what I originally had. Of course, that is the worse type and I have been on meds that do not help the other type which explains my downward spiral somewhat. I have now started PT and OT and they are putting in for a scooter (I got a walker already) so I can get around if I need to. May your health be good!"
)
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter